## **Análise de Demonstrações Financeiras - Embraer S.A. (2020 a 2024)**

In [17]:
# bibliotecas importadas
import requests
import zipfile
import os
import pandas as pd
import io
from openpyxl import Workbook

### Baixar as DFPs (2019–2023) da Embraer em XBRL diretamente do site da CVM

In [18]:
# ===== Configurações =====
ANOS = range(2021, 2025)  # 2020, 2021, 2022, 2023, 2024
URL_BASE = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_{}.zip"
EMPRESA = "EMBRAER S.A."
PASTA_ZIP = "zips"
PASTA_SAIDA = "saida"

# Nome-padrão dentro do zip para os demonstrativos consolidados
DEMONSTRATIVOS = {
    "Ativo": "BPA_con",
    "Passivo": "BPP_con",
    "DRE": "DRE_con",
    "DFC": "DFC_MI_con"
}

os.makedirs(PASTA_ZIP, exist_ok=True)
os.makedirs(PASTA_SAIDA, exist_ok=True)

dfs = {nome: [] for nome in DEMONSTRATIVOS.keys()}

for ano in ANOS:
    print(f"🔽 Baixando ano {ano}...")
    url = URL_BASE.format(ano)
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"❌ Não conseguiu baixar para o ano {ano}. Status: {resp.status_code}")
        continue

    # Salvar zip localmente opcionalmente
    caminho_zip = os.path.join(PASTA_ZIP, f"dfp_cia_aberta_{ano}.zip")
    with open(caminho_zip, "wb") as f:
        f.write(resp.content)
    print(f"✅ Zip salvo: {caminho_zip}")

    # Abrir zip em memória
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        nomes_zip = z.namelist()
        for nome_dem, padrao in DEMONSTRATIVOS.items():
            # encontrar arquivo CSV consolidado que contenha o padrão
            # ex: “dfp_cia_aberta_BPA_con_2023.csv”
            possiveis = [f for f in nomes_zip if padrao in f and f.endswith(".csv")]
            if not possiveis:
                print(f"⚠️ Não achei arquivo {padrao} para ano {ano}")
                continue

            nome_arq = possiveis[0]
            print(f"   ↳ Extraindo {nome_arq} para {nome_dem}, ano {ano}")

            with z.open(nome_arq) as arq:
                # Ler CSV com encoding e separador certos; pode ajustar dependendo do CSV
                df = pd.read_csv(arq, sep=";", encoding="latin1", dtype=str)

            # Filtrar para Embraer
            df_emb = df[df["DENOM_CIA"].str.strip().str.upper() == EMPRESA.upper()]
            if df_emb.empty:
                print(f"      ‼️ Nenhum registro para Embraer no {nome_dem}, ano {ano}")
            else:
                df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
                dfs[nome_dem].append(df_emb)

for nome_dem, lista in dfs.items():
    if not lista:
        print(f"⚠️ Sem dados para {nome_dem} nos anos especificados.")
        continue
    df_total = pd.concat(lista, ignore_index=True)
    caminho_saida = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    df_total.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=";")
    print(f"✅ Salvou {caminho_saida}")


🔽 Baixando ano 2021...
✅ Zip salvo: zips\dfp_cia_aberta_2021.zip
   ↳ Extraindo dfp_cia_aberta_BPA_con_2021.csv para Ativo, ano 2021
   ↳ Extraindo dfp_cia_aberta_BPP_con_2021.csv para Passivo, ano 2021


C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year


   ↳ Extraindo dfp_cia_aberta_DRE_con_2021.csv para DRE, ano 2021
   ↳ Extraindo dfp_cia_aberta_DFC_MI_con_2021.csv para DFC, ano 2021
🔽 Baixando ano 2022...


C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year


✅ Zip salvo: zips\dfp_cia_aberta_2022.zip
   ↳ Extraindo dfp_cia_aberta_BPA_con_2022.csv para Ativo, ano 2022
   ↳ Extraindo dfp_cia_aberta_BPP_con_2022.csv para Passivo, ano 2022


...

✅ Zip salvo: zips\dfp_cia_aberta_2023.zip
   ↳ Extraindo dfp_cia_aberta_BPA_con_2023.csv para Ativo, ano 2023
   ↳ Extraindo dfp_cia_aberta_BPP_con_2023.csv para Passivo, ano 2023


C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a 

   ↳ Extraindo dfp_cia_aberta_DRE_con_2023.csv para DRE, ano 2023
   ↳ Extraindo dfp_cia_aberta_DFC_MI_con_2023.csv para DFC, ano 2023
🔽 Baixando ano 2024...
✅ Zip salvo: zips\dfp_cia_aberta_2024.zip
   ↳ Extraindo dfp_cia_aberta_BPA_con_2024.csv para Ativo, ano 2024
   ↳ Extraindo dfp_cia_aberta_BPP_con_2024.csv para Passivo, ano 2024


C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_15616\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a 

   ↳ Extraindo dfp_cia_aberta_DRE_con_2024.csv para DRE, ano 2024
   ↳ Extraindo dfp_cia_aberta_DFC_MI_con_2024.csv para DFC, ano 2024
✅ Salvou saida\Ativo_Embraer_2020-2024.csv
✅ Salvou saida\Passivo_Embraer_2020-2024.csv
✅ Salvou saida\DRE_Embraer_2020-2024.csv
✅ Salvou saida\DFC_Embraer_2020-2024.csv


In [19]:
for nome_dem in DEMONSTRATIVOS.keys():
    caminho = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    if not os.path.exists(caminho):
        continue
    df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig", dtype=str)
    # Selecionar colunas relevantes (conferir futuramente)
    df_sel = df[["CD_CONTA", "DS_CONTA", "ANO", "VL_CONTA"]]
    # Pivotar para ter anos como colunas
    df_pivot = df_sel.pivot_table(index=["CD_CONTA", "DS_CONTA"], 
                                  columns="ANO", values="VL_CONTA", aggfunc="first").reset_index()
    
    novo = os.path.join(PASTA_SAIDA, f"{nome_dem}_contabil_2020-2024.csv")
    df_pivot.to_csv(novo, index=False, encoding="utf-8-sig", sep=";")
    print(f"📦 Salvou versão contabil: {novo}")

📦 Salvou versão contabil: saida\Ativo_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\Passivo_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\DRE_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\DFC_contabil_2020-2024.csv


In [20]:
ARQUIVO_XLSM = "Demonstrações_Embraer.xlsm"

arquivos = {
    "Ativo": "Ativo_Embraer_2020-2024.csv",
    "Passivo": "Passivo_Embraer_2020-2024.csv",
    "DRE": "DRE_Embraer_2020-2024.csv",
    "DFC": "DFC_Embraer_2020-2024.csv"
}

with pd.ExcelWriter(ARQUIVO_XLSM, engine="openpyxl") as writer:
    for aba, nome_arq in arquivos.items():
        caminho = os.path.join(PASTA_SAIDA, nome_arq)
        if not os.path.exists(caminho):
            print(f"⚠️ Arquivo não encontrado: {caminho}")
            continue

        df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")

        # Pivotar: Conta x Ano
        if "CD_CONTA" in df.columns and "DS_CONTA" in df.columns:
            df["Conta"] = df["CD_CONTA"] + " - " + df["DS_CONTA"]
        else:
            df["Conta"] = df["DS_CONTA"]

        # Converter valores para numérico (quando possível)
        df["VALOR"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")

        tabela = df.pivot_table(
            index="Conta",
            columns="ANO",
            values="VALOR",
            aggfunc="first"
        ).reset_index()

        tabela.to_excel(writer, sheet_name=aba, index=False)

print(f"✅ Arquivo Excel criado: {ARQUIVO_XLSM}")

✅ Arquivo Excel criado: Demonstrações_Embraer.xlsm


### Correção monetária pelo IPCA para 2024

In [21]:
# IPCA anual acumulado (coletado no https://www.yahii.com.br/ipca.html)
ipca = {
    2020: 0.0452,
    2021: 0.1006,
    2022: 0.0579,
    2023: 0.0462,
    2024: 0.0462
}

# Calcular fatores de correção para base 2024
fator_2024 = 1.0
fator_2023 = (1 + ipca[2024])
fator_2022 = (1 + ipca[2023]) * fator_2023
fator_2021 = (1 + ipca[2022]) * fator_2022
fator_2020 = (1 + ipca[2021]) * fator_2021

fatores = {
    2020: fator_2020,
    2021: fator_2021,
    2022: fator_2022,
    2023: fator_2023,
    2024: fator_2024
}

print("\nFatores de Correção para base 2024:")
for ano, fator in fatores.items():
    print(f"  - Ano {ano}: {fator:.4f}")



Fatores de Correção para base 2024:
  - Ano 2020: 1.2744
  - Ano 2021: 1.1579
  - Ano 2022: 1.0945
  - Ano 2023: 1.0462
  - Ano 2024: 1.0000


In [22]:
# Aplicar a correção a cada arquivo na pasta "saida"
for nome_arq in os.listdir(PASTA_SAIDA):
    if nome_arq.endswith(".csv"):
        caminho = os.path.join(PASTA_SAIDA, nome_arq)
        df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")
        
        if "ANO" in df.columns and "VL_CONTA" in df.columns:
            df["VALOR_CORRIGIDO_2024"] = df.apply(
                lambda x: pd.to_numeric(x["VL_CONTA"], errors="coerce") * fatores.get(int(x["ANO"]), 1),
                axis=1
            )
            
            # Salvar com os valores corrigidos
            nome_corrigido = nome_arq.replace(".csv", "_corrigido_2024.csv")
            df.to_csv(os.path.join(PASTA_SAIDA, nome_corrigido), sep=";", encoding="utf-8-sig", index=False)
            print(f"✅ Corrigido e salvo: {nome_corrigido}")


✅ Corrigido e salvo: Ativo_Embraer_2020-2024_corrigido_2024.csv
✅ Corrigido e salvo: Passivo_Embraer_2020-2024_corrigido_2024.csv
✅ Corrigido e salvo: DRE_Embraer_2020-2024_corrigido_2024.csv
✅ Corrigido e salvo: DFC_Embraer_2020-2024_corrigido_2024.csv
✅ Corrigido e salvo: Ativo_Embraer_2020-2024_corrigido_2024_corrigido_2024.csv
✅ Corrigido e salvo: Passivo_Embraer_2020-2024_corrigido_2024_corrigido_2024.csv
✅ Corrigido e salvo: DRE_Embraer_2020-2024_corrigido_2024_corrigido_2024.csv
✅ Corrigido e salvo: DFC_Embraer_2020-2024_corrigido_2024_corrigido_2024.csv


### Após correção com o IPCA de 2024


In [23]:
ARQUIVO_XLSM = "Demonstrações_Embraer_Ajustado.xlsm"

arquivos_corrigidos = {
    "Ativo": "Ativo_Embraer_2020-2024_corrigido_2024.csv",
    "Passivo": "Passivo_Embraer_2020-2024_corrigido_2024.csv",
    "DRE": "DRE_Embraer_2020-2024_corrigido_2024.csv",
    "DFC": "DFC_Embraer_2020-2024_corrigido_2024.csv"
}

with pd.ExcelWriter(ARQUIVO_XLSM, engine="openpyxl") as writer:
    for aba, nome_arq in arquivos_corrigidos.items():
        caminho = os.path.join(PASTA_SAIDA, nome_arq)
        if not os.path.exists(caminho):
            print(f"⚠️ Arquivo não encontrado: {caminho}")
            continue

        df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")

        # Converte valores corrigidos para numérico
        df["VALOR_CORRIGIDO_2024"] = pd.to_numeric(df["VALOR_CORRIGIDO_2024"], errors="coerce")

        # Pivotar: Conta x Ano (com valores já corrigidos)
        tabela_corrigida = df.pivot_table(
            index=["CD_CONTA", "DS_CONTA"],
            columns="ANO",
            values="VALOR_CORRIGIDO_2024",
            aggfunc="first"
        ).reset_index()

        # Exporta cada aba para o Excel
        tabela_corrigida = tabela_corrigida.sort_values(by="CD_CONTA")
        tabela_corrigida.to_excel(writer, sheet_name=aba, index=False)

print(f"Arquivo Excel final criado com sucesso: {ARQUIVO_XLSM}")
print("Todas as guias foram corrigidas e consolidadas com base no IPCA de 2024.")


Arquivo Excel final criado com sucesso: Demonstrações_Embraer_Ajustado.xlsm
Todas as guias foram corrigidas e consolidadas com base no IPCA de 2024.
